Libraries

In [14]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

Data Preprocessing

In [12]:
# Import training set
train_path = 'D:/Onedrive/experiments/experiments/ML\DL/P16-Deep-Learning-AZ/Deep Learning A-Z/Part 3 - Recurrent Neural Networks (RNN)/Google_Stock_Price_Train.csv'
dataset_train = pd.read_csv(train_path)
training_set = dataset_train.iloc[:, 1:2].values # only the 'open' column

# Feature Scaling
# its recommended to use normalization when sigmoid is used as activation function
sc = MinMaxScaler(feature_range=(0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Creating a data structure with 60 time steps and 1 output
# basically the last 60 time steps of data will be used to predict the next output
x_train = [] # last 60 stock prices
y_train = [] # next output
# also means we have to start predicting from 60th day onwards and not from day 1
for i in range(60, 1258):
    x_train.append(training_set_scaled[i - 60: i, 0])
    y_train.append(training_set_scaled[i, 0])

x_train, y_train = np.array(x_train), np.array(y_train)

# Reshaping the data to include more params later (if required)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))


Building the RNN

In [16]:
# Building a stacked LSTM with dropouts to prevent overfitting
# Initialize the RNN
regressor = Sequential()
# Adding 1st LSTM Layer and some dropout regularization
regressor.add(LSTM(units = 64, return_sequences = True, input_shape=(x_train.shape[1], 1))) # (no of units, return sequence, input shape)
# Since we're using a stacked LSTM, we need to set return sequence to 'true'
regressor.add(Dropout(0.2)) # 20% dropout (recommended value)
# Adding 2nd LSTM Layer and some dropout regularization
regressor.add(LSTM(units = 64, return_sequences = True)) # (no of units, return sequence, input shape)
regressor.add(Dropout(0.2)) # 20% dropout (recommended value)
# Adding 3rd LSTM Layer and some dropout regularization
regressor.add(LSTM(units = 128, return_sequences = True)) # (no of units, return sequence, input shape)
regressor.add(Dropout(0.2)) # 20% dropout (recommended value)
# Adding 4th LSTM Layer and some dropout regularization (last LSTM layer so return sequence must be false)
regressor.add(LSTM(units = 64)) # (no of units, return sequence, input shape)
regressor.add(Dropout(0.2)) # 20% dropout (recommended value)
# Adding Output Layer
regressor.add(Dense(units = 1)) # Output is the stock price for the next time step (single output)

In [17]:
# Compiling the RNN
# usually RMSprop optimizer is suggested for RNN
regressor.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Fitting the RNN to the training set
regressor.fit(x_train, y_train, epochs = 100, batch_size = 32)

Making Predictions and Visualization